In [1]:
# Importar todo de la clase socket
from socket import *

In [3]:
serverSocket = socket(AF_INET, SOCK_STREAM)
# Creo un objeto de tipo Socket

# Puerto del Socket
serverPort = 8888

# IP a la que bindear el puerto, si el str esta vacío lo bindea al localhost
host = ""
serverSocket.bind((host, serverPort))

# Comenzar a escuchar 1 sola conexion por vez
# Listen acepta por parametro la cantidad de conexiones concurrentes
serverSocket.listen(1)

In [ ]:
# Aceptar infinitas conexiones secuenciales
while True:
  print('Sirviendo')
  
  # Aceptar una conexion, devuelve una tupla con un objeto conexion y la ip del host que se conecto
  connectionSocket, addr = serverSocket.accept()
  print('Se conectó la IP', addr)

  # Si falla salta la exception y devuelve un error al host conectado
  try:
    # Recibe el mensaje en bits
    message = connectionSocket.recv(1024)
    print("msg", message)

    # Splitea la request por caracteres espacios. 
    # El primer elemento de la lista spliteada es el tipo de request
    # El segundo es el archivo
    # Ejemplo: GET, index.html
    filename = message.split()[1]
    print("filename", filename)
    
    # Omito la respuesta si la request es al favicon (mas facil)
    if filename != b'/favicon.ico':
      
      # Abro el archivo
      f = open(filename[1:])

      # Leo el string del archivo
      outputdata = f.read()
      #print("file", outputdata)
      
      # Header de la Response a enviar. 
      # Especifica protocolo HTTP y versión 1.1, defaultea a 0.9 (ya no se lo bancan algunos browsers)
      # Status 200 es que está OK
      header = 'HTTP/1.1 200 OK\r\n\r\n'
      # Encodea el string del header a bits y lo envia al host conectado
      connectionSocket.send(header.encode())

      # Encodea el string del archivo leido a bits y lo envia...
      connectionSocket.send(outputdata.encode())
  except IOError:
    # Devuelve header de error (404 es file not found, https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
    connectionSocket.send("HTTP/1.1 404 File not found\r\n\r\n")
  
  # Cierro la conexion
  connectionSocket.close()
  # Vuelve a iterar el While loop para recibir otra conexión en el mismo socket.

Sirviendo
Se conectó la IP ('10.9.12.32', 58587)
msg b'GET /taller.html HTTP/1.1\r\nHost: 10.9.14.206:8888\r\nConnection: keep-alive\r\nCache-Control: max-age=0\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\r\nAccept-Encoding: gzip, deflate\r\nAccept-Language: es-ES,es;q=0.9\r\n\r\n'
filename b'/taller.html'
Sirviendo
Se conectó la IP ('10.9.12.32', 58588)
msg b'GET /favicon.ico HTTP/1.1\r\nHost: 10.9.14.206:8888\r\nConnection: keep-alive\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36\r\nAccept: image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8\r\nReferer: http://10.9.14.206:8888/taller.html\r\nAccept-Encoding: gzip

In [ ]:
# Matar el socket
serverSocket.close()